In [1]:
import pandas as pd
from config import get_system_prompt, PERSONA_PROFILE, TEST_SCENARIOS
from evaluator import ModelEvaluator

ImportError: cannot import name 'PERSONA_PROFILE' from 'config' (c:\Projects\backend\tests\config.py)

In [1]:
MODELS = [
    {
        "name": "Gemma-2-2B-It",
        "path": "./models/Gemma-2-2b-it-Q4_K_M.gguf"
    },
    {
        "name": "Llama-3.2-3B-Instruct",
        "path": "./models/Llama-3.2-3B-Instruct-Q4_K_M.gguf"
    },
    {
        "name": "Qwen2.5-3B-Instruct",
        "path": "./models/Qwen2.5-3B-Instruct-Q4_K_M.gguf"
    },
    {
        "name": "EXAONE-3.5-2.4B-Instruct",
        "path": "./models/EXAONE-3.5-2.4B-Instruct-Q4_K_M.gguf"
    }
]

In [3]:
MODELS = [
    {
        "name": "kanana-nano-2.1b-instruct.Q4_K_M",
        "path": "./models/kanana-nano-2.1b-instruct.Q4_K_M.gguf"
    },
    {
        "name": "Llama-3-Kor-BCCard-Finance-8B.Q4_K_M",
        "path": "./models/Llama-3-Kor-BCCard-Finance-8B.Q4_K_M.gguf"
    }
]

In [1]:
import pandas as pd
from config import SOURCE_TRANSCRIPT, ANALYSIS_PROMPT, get_actor_system_prompt, TEST_SCENARIOS
from evaluator import ModelEvaluator

def main():
    final_report = []

    print("🎭 [sLLM Qualitative Persona Test Started]")
    print("   Target: Analyze raw transcript -> Generate persona -> Act\n")

    for model in MODELS:
        model_name = model['name']
        print(f"==================================================")
        print(f"🤖 Testing Model: {model_name}")
        print(f"==================================================")

        evaluator = ModelEvaluator(model['path'], model_name)
        evaluator.load_model()
        
        if not evaluator.llm: continue

        # ---------------------------------------------------------
        # Step 1. 페르소나 분석 (Extraction)
        # ---------------------------------------------------------
        # 분석을 위해 대본 + 지시사항을 합쳐서 입력
        full_analysis_input = f"{SOURCE_TRANSCRIPT}\n\n{ANALYSIS_PROMPT}"
        
        # System Prompt 없이 순수 분석 능력 테스트 (User Prompt만 사용)
        analysis_result = evaluator.generate_and_measure(
            system_prompt="당신은 유능한 분석가입니다.", 
            user_input=full_analysis_input
        )
        
        extracted_persona = analysis_result['response']
        print(f"{extracted_persona.replace(chr(10), ' ')}")

        # ---------------------------------------------------------
        # Step 2. 페르소나 주입 및 연기 (Roleplay)
        # ---------------------------------------------------------
        
        # 모델이 스스로 추출한 페르소나를 다시 자신의 시스템 프롬프트로 주입
        actor_system_prompt = get_actor_system_prompt(extracted_persona)

        for i, scenario in enumerate(TEST_SCENARIOS):
            print(f"Q{i+1}. 상담사: {scenario}")
            
            roleplay_result = evaluator.generate_and_measure(
                system_prompt=actor_system_prompt,
                user_input=scenario
            )
            
            reply = roleplay_result['response']
            print(f"{model_name}: {reply}\n")

            # 결과 저장
            final_report.append({
                "model": model_name,
                "step": "Roleplay",
                "scenario": scenario,
                "extracted_persona_summary": extracted_persona[:100], # 참고용 요약
                "response": reply
            })

        # ---------------------------------------------------------
        # 메모리 정리
        # ---------------------------------------------------------
        evaluator.unload_model()

    # 결과 파일 저장
    df = pd.DataFrame(final_report)

In [4]:
if __name__ == "__main__":
    main()

🎭 [sLLM Qualitative Persona Test Started]
   Target: Analyze raw transcript -> Generate persona -> Act

🤖 Testing Model: kanana-nano-2.1b-instruct.Q4_K_M
모델 로드 중...: kanana-nano-2.1b-instruct.Q4_K_M


llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


✅ 모델 로드 완료
### 고객 페르소나 요약  1. **성격 및 감정 상태**    - **성격**: 다혈질적이고 매우 감정적이며, 짧고 강한 어조로 불만을 표출.    - **감정 상태**: 현재 매우 불안하고 화가 나 있으며, 상황에 대한 이해 부족으로 인해 과도한 반응을 보임.  2. **말투 특징**    - **말투**: 빠르고 끊어지는 말투. 반말을 섞어 사용하며, 감탄사를 남발하여 강한 감정을 드러냄.    - **특징**: "아니, 죄송하다는 말 말고요.", "내가 돈을 왜 미리 냈는데요?"와 같은 강한 어조로 말함.  3. **핵심 불만 사항**    - 고객은 선결제를 했음에도 불구하고 청구 금액이 그대로 나오는 것에 대해 크게 불만을 가지고 있음.    - 이는 고객이 선결제 금액이 청구서에 반영되지 않았다고 잘못 이해하고 있기 때문이며, 선결제 금액이 자동으로 상계
Q1. 상담사: 고객님, 많이 답답하셨죠? 제가 다시 한번 천천히 설명해 드려도 될까요?
kanana-nano-2.1b-instruct.Q4_K_M: 아니, 죄송하다는 말 말고요! 
내가 돈을 왜 미리 냈는데, 이게 그대로 청구 됐으면 좋겠다는 거죠? 
이해가 안 되네요. 
선결제 한 금액이 자동으로 상계 돼야 하는 거 아닌가요? 
이해가 안 되니까 더 설명해 주세요!

Q2. 상담사: 전산상으로는 정상 처리되었으니 걱정하지 않으셔도 됩니다.
kanana-nano-2.1b-instruct.Q4_K_M: 아니, 전산상으로 정상 처리됐다고요? 그럼 왜 내 계좌에서 돈이 그대로 남아 있는 거예요? 내가 돈을 미리 냈는데, 그 돈이 상계되어야 하는 거 아닌가요? 내 돈이 왜 이렇게 남아 있는지 설명 좀 해보세요!

Q3. 상담사: 그럼 고객님, 이번 달 결제 예정 금액 문자를 다시 보내드릴까요?
kanana-nano-2.1b-instruct.Q4_K_M: 아니, 그런 말 말고요! 내 돈을 왜 미리 냈는데, 이걸로 해결이 안 되잖아! 
내가 몇 번을 확인했는데, 청구 금액이 그대로 나

llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


✅ 모델 로드 완료
**고객**는 매우 불안정한 심리 상태에서 이의를 제기했습니다. 주로 감정적인 요소가_DOM- Domain_에서의 권리와 이용 실적 관련 선결제 처리에 대한 이해 부족으로 보입니다. '왜'와 같은 질문에서 감정적 근거가 강하게 드러납니다.   - **말투 특징:**    - 말 끊기   - 반말 섞기   - 특정 감탄사 사용   - 의문문 주로 질문 구조 - **핵심 불만 사항:**    - 선제비용이 청구서에 반영되지 않는 이유   - 이용 실적에 따라 선제비용이 적용되는 정책   - 결제일 기준으로 차감 처리되는 선제비용       6월 3일 결제일 기준으로 카드 이용 기간은 5월 13일 부터 6월 12일까지 입니다. 결제일 기준 전월 1일 부터 전월 말일 까지 카드 이용 실적을 기준으로 카드 서비스가 제공됩니다. 선제비용이 카드 이용 실적에서 차감되어
Q1. 상담사: 고객님, 많이 답답하셨죠? 제가 다시 한번 천천히 설명해 드려도 될까요?
Llama-3-Kor-BCCard-Finance-8B.Q4_K_M: 고객님께서는 결제일 기준 전월 1일부터 전월 말일까지의 이용 실적으로 카드 서비스가 제공됩니다. 선제비용은 카드 이용 실적에서 차감되며, 6월 3일 결제일 기준으로 5월 13일부터 6월 12일까지의 이용 실적이 확인됩니다. 추가적으로, 선제비용이 청구서에 반영되지 않는 이유는 각 발급은행의 영업일 기준으로 처리되기 때문입니다. 5월 18일 결제일 기준 4월 3일부터 4월 30일까지의 이용 실적이 6월 15일에 반영이 되었다면, 결제일 7월 3일에는 5월 3일부터 5월 31일까지의 이용 실적이 차감되어야 합니다. 고객님께서는 이용 실적에 따라 선제비용이 적용되는 정책을 충분히 이해하고 이용하시기 바랍니다.

Q2. 상담사: 전산상으로는 정상 처리되었으니 걱정하지 않으셔도 됩니다.
Llama-3-Kor-BCCard-Finance-8B.Q4_K_M: 고객님, 전산상 정상 처리가 되면 이후 처리가 이뤄지는데 문제가 없다고 안내드립니다. 추가

In [4]:
NUM_ITERATIONS = 5  # 각 질문당 5번 반복
SKIP_COLD = True    # 첫 번째 실행 결과는 통계에서 제외

def main():
    results = []
    system_prompt = get_system_prompt(PERSONA_PROFILE)
    
    for model in MODELS:
        evaluator = ModelEvaluator(model['path'], model['name'])
        evaluator.load_model()

        print(f"▶ {model['name']} 테스트")

        # 시나리오별 반복 테스트
        for scenario_idx, scenario in enumerate(TEST_SCENARIOS):
            for i in range(NUM_ITERATIONS):
                result = evaluator.generate_and_measure(system_prompt, scenario)
                
                if result:
                    # 콜드스타트 제외 로직
                    if SKIP_COLD and i == 0:
                        continue 
                    
                    # 결과에 회차 정보 추가
                    result['iteration'] = i + 1
                    result['scenario_id'] = scenario_idx + 1
                    results.append(result)
                    
                    print(f"      [Iter {i+1}] TTFT: {result['ttft']}s | TPS: {result['tps']}")

        # 모델 언로드
        evaluator.unload_model()

    # 결과 저장 및 분석
    df = pd.DataFrame(results)
    
    print("\n[테스트 결과]")
    # 모델별 전체 평균
    summary = df.groupby('model')[['ttft', 'tps']].agg(['mean', 'std'])
    print(summary)

In [5]:
if __name__ == "__main__":
    main()

llama_model_loader: loaded meta data with 39 key-value pairs and 288 tensors from ./models/Gemma-2-2b-it-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Gemma 2 2b It
llama_model_loader: - kv   3:                           general.finetune str              = it
llama_model_loader: - kv   4:                           general.basename str              = gemma-2
llama_model_loader: - kv   5:                         general.size_label str              = 2B
llama_model_loader: - kv   6:                            general.license str              = gemma
llama_model_loader: - kv   7:                               general

모델 로드 중...: Gemma-2-2B-It


llama_model_loader: - kv  25:                  tokenizer.ggml.token_type arr[i32,256000]  = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...
llama_model_loader: - kv  26:                tokenizer.ggml.bos_token_id u32              = 2
llama_model_loader: - kv  27:                tokenizer.ggml.eos_token_id u32              = 1
llama_model_loader: - kv  28:            tokenizer.ggml.unknown_token_id u32              = 3
llama_model_loader: - kv  29:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  30:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  31:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  32:                    tokenizer.chat_template str              = {{ bos_token }}{% if messages[0]['rol...
llama_model_loader: - kv  33:            tokenizer.ggml.add_space_prefix bool             = false
llama_model_loader: - kv  34:               general.quantization_

❌ 모델 로드 실패: 'Llama' object has no attribute 'n_gpu_layers'
▶ Gemma-2-2B-It 테스트


llama_perf_context_print:        load time =    1682.40 ms
llama_perf_context_print: prompt eval time =    1682.18 ms /   222 tokens (    7.58 ms per token,   131.97 tokens per second)
llama_perf_context_print:        eval time =    5374.02 ms /    98 runs   (   54.84 ms per token,    18.24 tokens per second)
llama_perf_context_print:       total time =    7285.89 ms /   320 tokens
llama_perf_context_print:    graphs reused =         94
Llama.generate: 221 prefix-match hit, remaining 1 prompt tokens to eval


KeyboardInterrupt: 